In [5]:
import tkinter as tk
from tkinter import scrolledtext
from transformers import pipeline
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import speech_recognition as sr
import pyttsx3
import pandas as pd

model_checkpoint2 = "aditi2212/roberta-finetuned-ChennaiQA-final"
question_answerer = pipeline("question-answering", model=model_checkpoint2)
model = SentenceTransformer('all-mpnet-base-v2')
df_train=pd.read_csv(r'C:\Users\vidya\OneDrive\Desktop\FYP\Train_data_final.csv')
df_test=pd.read_csv(r'C:\Users\vidya\OneDrive\Desktop\FYP\Test_data_final.csv')
frames = [df_train, df_test]
df_total = pd.concat(frames)

unique_context = pd.DataFrame(df_total['review'].unique())
unique_context.rename( columns={0:'Context'}, inplace=True )
contexts_list = unique_context['Context'].tolist()

In [6]:
def Speech_to_text():
    r = sr.Recognizer()
    with sr.Microphone() as source:
        #print("How can I help you?")
        audio = r.listen(source)
    try:
        text = r.recognize_google(audio)
        return text
    except sr.UnknownValueError:
        print("Sorry, I didn't catch that. Could you please repeat?")
    except sr.RequestError as e:
        print("Could not request results; {0}".format(e))

In [7]:
def text_to_speech(my_answer):
    engine = pyttsx3.init() # object creation
    engine.setProperty('rate', 200)
    voices = engine.getProperty('voices')
    # chnging index, changes voices. 1 for femals
    engine.setProperty('voice', voices[1].id)

    engine.say(my_answer)
    engine.runAndWait()

In [8]:
#model = SentenceTransformer('all-mpnet-base-v2')
def pickRelevantContext(question, contexts_list):
    question_embedding = model.encode(question)
    context_embeddings = model.encode(contexts_list)
    similarity_scores = cosine_similarity([question_embedding], context_embeddings)
    most_similar_index = similarity_scores.argmax()
    #print("Most relevant context:", contexts_list[most_similar_index])
    #print("Relevance score:", similarity_scores[0][most_similar_index])
    context=contexts_list[most_similar_index]
    return context

In [9]:
def on_click_text():
    #display_initial_message()
    #display_bot_message("How can I help you?")
    display_loading_message()
    question = text_box.get("1.0", "end-1c")
    most_relevant_context = pickRelevantContext(question, contexts_list)
    my_answer = question_answerer(question=question, context=most_relevant_context)['answer']
    
    display_question(question)
    display_answer(my_answer)
    text_to_speech(my_answer)

def on_click_speech():
    #display_initial_message()
    #display_bot_message("How can I help you?")
    display_speaknow_message()
    question = Speech_to_text()
    most_relevant_context = pickRelevantContext(question, contexts_list)
    my_answer = question_answerer(question=question, context=most_relevant_context)['answer']
    #display_bot_message("How can I help you?")
    display_loading_message()
    display_question(question)
    display_answer(my_answer)
    text_to_speech(my_answer)

def display_initial_message():
    chat_history.config(state=tk.NORMAL)
    chat_history.insert(tk.END, "Bot: How can I help you?\n\n", "bot_message")
    chat_history.config(state=tk.DISABLED)
    chat_history.see(tk.END)
    
def display_loading_message():
    chat_history.config(state=tk.NORMAL)
    chat_history.insert(tk.END, "Bot: Loading...\n\n", "bot_message")
    chat_history.config(state=tk.DISABLED)
    chat_history.see(tk.END)
    
def display_speaknow_message():
    chat_history.config(state=tk.NORMAL)
    chat_history.insert(tk.END, "Bot: Speak Now...\n\n", "bot_message")
    chat_history.config(state=tk.DISABLED)
    chat_history.see(tk.END)


def display_answer(answer):
    chat_history.config(state=tk.NORMAL)
    chat_history.insert(tk.END, "Answer: " + answer + "\n\n", "user_message")
    chat_history.config(state=tk.DISABLED)
    chat_history.see(tk.END)

def display_question(question):
    chat_history.config(state=tk.NORMAL)
    chat_history.insert(tk.END, "Question: " + question + "\n\n", "user_message")
    chat_history.config(state=tk.DISABLED)
    chat_history.see(tk.END)

# GUI
root = tk.Tk()
root.title("Chennai Tourism Chatbot")
root.geometry("500x500")
root.configure(bg='black')

chat_history = scrolledtext.ScrolledText(root, width=40, height=15, bg='black', fg='white', wrap=tk.WORD)
chat_history.tag_config("bot_message", foreground="blue")
chat_history.tag_config("user_message", foreground="white")
chat_history.pack(pady=10)
display_initial_message()
text_box = tk.Text(root, width=40, height=3, bg='black', fg='white')
text_box.pack(pady=10)

text_button = tk.Button(root, text="Submit Text", command=on_click_text, bg='blue', fg='white')
text_button.pack()

speech_button = tk.Button(root, text="Speech Input", command=on_click_speech, bg='blue', fg='white')
speech_button.pack()

root.mainloop()